# Обучение с подкреплением

Кузин Мирослав 4.6

# Траектория:
$$\tau = (s_0,a_0,r_0),\dots,(s_T,a_T,r_T) ,$$
где $s_i$ состояние среды на шаге $i$, $a_i$ воздействие агента на среду на шаге $i$,
$r_i$ вознаграждение на шаге $i$.

Вознаграждение начиная с шага $t$:
$$R_t(\tau)=\sum\limits_{j=t}^{T}\gamma^{j-t}r_{j}, $$
$$R(\tau) = R_0(\tau) $$
где $\gamma$ дисконтный множитель (коэффициент значимости вознаграждения при переходе на следущий шаг).

Обучение с подкреплением - набор методов, позволяющих агенту (интеллектуальной системе) вырабатывать оптимальную стратегию при его взаимодействии со средой (внешним миром). 

Среда предоставляет информацию, описывающую состояние системы. Агент взаимодействует со средой, наблюдая состояние и используя данную информацию при выборе действия. Среда принимает действие и переходит в следующее состояние, а затем возвращает агенту следующее состояние и вознаграждение. Когда цикл «состояние → действие → вознаграждение» завершен, предполагается, что сделан один шаг. Цикл повторяется, пока среда не завершится, например, когда задача решена.

Стратегия $\pi$ — это функция, отображающая состояния вероятности действий, которые используются для выбора действия $a \sim \pi(s) $. 

Стратегия $\pi$ содержит настраеваемые параметры $\omega$, чтобы это подчеркнуть будем писать $\pi_{\omega}(s)$.

Целевая функция — это ожидаемая отдача по всем полным траекториям, порожденным агентом.
$$J(\omega)=J(\pi_{\omega})=M_{\tau\sim \omega}R(\tau),$$
где $M_{\tau\sim \omega}$ математическое ожидание по всех траекториям $\tau$ соответствующим значениям параметров $\omega$.

Задача:
$$J(\omega)\to \max $$

$$Loss(\omega) = - R(\tau)\sum\limits_t \ln\pi_{\omega}(a_t\mid s_t) $$

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os

work_dir = "./drive/MyDrive/Colab Notebooks/ml4course_1laba/" # Это все для колаба, директория на моем диске

from google.colab import files
src = list(files.upload().values())[0]
open('mancala.py','wb').write(src) # загружаем файл с игрой для импорта


Saving mancalaa.py to mancalaa.py


6204

In [1]:
from mancala import Kalah
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import time
import random

**Болваны**

In [2]:
# Бот простой, рандомно выбирает не нулевые элементы
def do_simple_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


# Бот МАСТЕР  захвата
def do_prof_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    state_copy = state.clone()
    captured = []
    additional_moves = []
    for cursor in nonzero_state_indexs:
        cursor = int(cursor)
        n = int(cursor)
        temp = int(state_copy[cursor])
        state_copy[cursor] = 0
        while temp > 0:
            temp -= 1
            if cursor in range(0, 6):
                cursor += 1
            elif cursor == 6:
                cursor = 13
            elif cursor in range(9, 14):
                cursor -= 1
            elif cursor == 8:
                cursor = 0

            state_copy[cursor] += 1

        if cursor == 6:
            additional_moves.append(n)

        if cursor < 6 and state_copy[cursor] == 1 and state_copy[cursor%6-6] != 0:
            state_copy[6] += sum([state_copy[cursor], state_copy[cursor%6-6]])
            captured.append((n, sum([state_copy[cursor], state_copy[cursor%6-6]])))
        state_copy = state.clone()


    if captured != []:
        rez_ind = sorted(captured, key=lambda x: x[1])
        rez = rez_ind[-1][0]
    # elif additional_moves != []: # не использовал(а) при обучении, ибо плохо обучается с этой фичей (проверка на доп ход)
    #     rez = additional_moves[torch.randint(0, len(additional_moves), (1,))[0]]
    else:
        rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


do_simple_bot_step(torch.Tensor([1, 0, 1, 0, 0, 0, 0]))
print(do_prof_bot_step(torch.Tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])))

tensor(4)


**Модель**

In [30]:
model = torch.load("model2_winner_test6.pt")

model2 = torch.load("model_winner_ver4.pt")
model2.to(torch.float)
model.to(torch.float)

Sequential(
  (0): Linear(in_features=14, out_features=28, bias=True)
  (1): ReLU()
  (2): Linear(in_features=28, out_features=56, bias=True)
  (3): ReLU()
  (4): Linear(in_features=56, out_features=6, bias=True)
  (5): Softmax(dim=-1)
)

In [31]:
from torchsummary import summary

# print(list(model.parameters())[0])
# print(f"model 2 {list(model2.parameters())}")
# print(torch.Tensor(list(model.parameters())[0])-torch.Tensor(list(model2.parameters())[0]))
print(torch.var(torch.Tensor(list(model.parameters())[1])), torch.mean(torch.Tensor(list(model.parameters())[1])))
print(torch.var(torch.Tensor(list(model2.parameters())[1])), torch.mean(torch.Tensor(list(model2.parameters())[1])))
print(summary(model, torch.Tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11, 12, 13]).size()))
print(summary(model2, torch.Tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11, 12, 13]).size()))

tensor(0.0267, grad_fn=<VarBackward0>) tensor(-0.0176, grad_fn=<MeanBackward0>)
tensor(0.0296, grad_fn=<VarBackward0>) tensor(0.0199, grad_fn=<MeanBackward0>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 28]             420
              ReLU-2                   [-1, 28]               0
            Linear-3                   [-1, 56]           1,624
              ReLU-4                   [-1, 56]               0
            Linear-5                    [-1, 6]             342
           Softmax-6                    [-1, 6]               0
Total params: 2,386
Trainable params: 2,386
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------
None
----------

In [15]:
model2.to(torch.float)

Sequential(
  (0): Linear(in_features=14, out_features=42, bias=True)
  (1): ReLU()
  (2): Linear(in_features=42, out_features=6, bias=True)
  (3): Softmax(dim=-1)
)

**Настройка обучения**

In [ ]:
def loss_func(probs: torch.Tensor, action: int, m: Categorical, R: int, n: int):
    alpha = 7 # 1 - при начале обучения, чтобы не ловить мега минус
    beta = 1e-4
    return -(alpha/n)*R*m.log_prob(action) # m.log_prob - обычный ln от вероятности действия a (m-объект Categorical, хранит в себе вероятности), т.е. ln a

# Начало обучения при lr=1.0e-3, после 10000 эпох 1.0e-4 там 30000 и до нынешнего ушло около 150000 эпох
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-6, amsgrad=True)

# Не использовалась, ибо на cpu уверенно шло
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


**Обучение**

In [48]:
# Проверка работы модели
game = Kalah()
print(game.get_general_state())
model(game.get_general_state().to(torch.float))

tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])


tensor([1.0456e-03, 9.9873e-01, 9.2770e-08, 2.2892e-04, 3.7919e-08, 1.1613e-07],
       grad_fn=<SoftmaxBackward0>)

In [13]:
def take_step_neuronet(game: Kalah):
    global model2
    old_player_making_step = game.get_player_making_step()
    # print("Ход нейронки! ", old_player_making_step == game.get_player_making_step())
    bad_choise = []
    while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
        # Выбор хода
        probs = model2(game.get_state().to(torch.float))
        action = probs.argmax()

        while action in bad_choise:
            probs[probs.argmax()] = 0
            action = probs.argmax()

            rezult_step = game.take_step(action + 1)
        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            print("Neuronet_emeny took step, but heap is empty", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            bad_choise.append(action)
        else:
            # print(f"Нейросеть делает ход! Выбор лунки {action + 1}")
            # game.print_state()
            bad_choise = []
        # print(action + 1)

*Награды и штрафы*

In [ ]:
rewards = torch.Tensor([15, 4, -300])

In [50]:
def take_step_bot(game: Kalah):
      old_player_making_step = game.get_player_making_step()
      while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
          # Выбор режима для бота
          # bot_action = do_prof_bot_step(game.get_state())
          bot_action = do_simple_bot_step(game.get_state())
          rezult_step = game.take_step(bot_action)

In [ ]:
game = Kalah()
episodes_count = 40000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
gamma = 0.85 # Регулируем от 0.9 до 1, можно меньше, если хотим больше учитывать текущий ход, а не игру в целом
check_optim = False # Для регулировки lr
check_optim2 = False
# reward = rewards[0]
for episode in range(0, episodes_count):
    game.set_new_game() # сброс игры
    tr = [] # массив хранения состояний всей игры [(игровое поле, действие при нем, награда за действие, объект Categorical (для лучшей работы ln))]

    # Выбор хода
    if episode % 2: #!= -1: 
        neuronet_walker_queue = 2
        # take_step_neuronet(game)
        take_step_bot(game)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.float)) # Модель вернула вероятности 
        m = Categorical(probs) # Создали объект для работы с ln и выбором действий с вероятностью
        action = m.sample() # Выбор действия согласно вероятности
        # print(action.dtype)  

        # Выбор награды и выполнение хода
        # Сохраняем старые значения, ибо если выберет ноль, действие повторятся или нет не известно, нужно преверять
        old_state = game.get_state().to(torch.float)
        old_score = game.get_score_player(neuronet_walker_queue)
        old_player_making_step = game.get_player_making_step()

        # Делаем ход
        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            reward = rewards[2]
            
            # game.set_is_game_over(True) # Использовалась при начале обучения. Если выбрал ноль - игра окончена.
        elif "Хороший ход!" in rezult_step:
            # reward = game.get_score_player(neuronet_walker_queue) - old_score # начисления без учета выигрыша камней противника
            reward = game.get_score_player(neuronet_walker_queue) - game.get_score_player(3 - neuronet_walker_queue) # начисление с учетом камней противника
            # print(rezult_step)
            if "Дополнительный ход" in rezult_step:
                reward = reward + rewards[1]
                # print("kek")

        # Сохраняем статистику за ход                
        tr.append([old_state, action, reward, m])

        # Ход бота/нейронки
        if old_player_making_step != game.get_player_making_step():
            # take_step_neuronet(game)
            take_step_bot(game)

    # Награждение за победу начисляется на последнем ходу
    if game.get_winner() != None:
        if neuronet_walker_queue == game.get_winner():
            count_win_neuronet += 1
            # rwrd = game.get_general_state()[5+neuronet_walker_queue] - game.get_general_state()[8-neuronet_walker_queue]
        rwrd = rewards[0] if neuronet_walker_queue == game.get_winner() else -rewards[0] + 8
        tr[-1][-2] = rwrd
    
    # Считаем награждения и лосс
    loss = 0.
    count_played_step = len(tr) # Кол-во сыгранных ходов
    for id_current_step in range(count_played_step):
        R = 0.
        for id_next_step in range(id_current_step, count_played_step):
            # R += (gamma**(id_current_step-id_next_step))*(tr[id_next_step][2] + rwrd)
            R += (gamma**(id_next_step-id_current_step))*tr[id_next_step][2]
            # R += (gamma**(id_current_step - id_next_step))*tr[id_next_step][2]
        loss += loss_func(model(tr[id_current_step][0]), tr[id_current_step][1], tr[id_current_step][3], R, count_played_step)

    if not check_optim and episode > 10000:
        check_optim = True
        for g in optimizer.param_groups:
            g['lr'] = 1.0e-7
        print("updated loss")
    # if not check_optim2 and episode > 50000:
    #     check_optim2 = True
    #     for g in optimizer.param_groups:
    #         g['lr'] = 1.0e-5
    #     print("updated loss")

    if not episode % 150 or episode %150 == 1:
        print(episode, loss, "Номер хода нейронки", neuronet_walker_queue, ", Победитель", game.get_winner(), tr[-1][-2], len(tr))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 tensor(17.1441, grad_fn=<AddBackward0>) Номер хода нейронки 1 , Победитель 1 tensor(15.) 15
1 tensor(-51.8400, grad_fn=<AddBackward0>) Номер хода нейронки 2 , Победитель 2 tensor(15.) 22
150 tensor(61.4358, grad_fn=<AddBackward0>) Номер хода нейронки 1 , Победитель 1 tensor(15.) 29
151 tensor(20.4449, grad_fn=<AddBackward0>) Номер хода нейронки 2 , Победитель 2 tensor(15.) 37
300 tensor(14.7412, grad_fn=<AddBackward0>) Номер хода нейронки 1 , Победитель 1 tensor(15.) 19
301 tensor(7.1277, grad_fn=<AddBackward0>) Номер хода нейронки 2 , Победитель 2 tensor(15.) 20
450 tensor(11.6027, grad_fn=<AddBackward0>) Номер хода нейронки 1 , Победитель 1 tensor(15.) 24
451 tensor(34.3777, grad_fn=<AddBackward0>) Номер хода нейронки 2 , Победитель 2 tensor(15.) 25
600 tensor(31.7070, grad_fn=<AddBackward0>) Номер хода нейронки 1 , Победитель 1 tensor(15.) 27
601 tensor(33.0397, grad_fn=<AddBackward0>) Номер хода нейронки 2 , Победитель 2 tensor(15.) 27
750 tensor(88.7853, grad_fn=<AddBackward0>) 

In [ ]:
print(check_optim)
print(count_choisen_zero)

True
176


In [ ]:
import pickle

**Тестирование модели**

In [55]:
game = Kalah()
episodes_count = 1000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
bad_choise = []
zero_list_state = []
rezult_step = "Хороший ход!"

for episode in range(0, episodes_count):
    game.set_new_game()
    if episode % 2:
        # old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        # take_step_neuronet(game)
        take_step_bot(game)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.float))
        action = probs.argmax()

        while action in bad_choise:
            probs[probs.argmax()] = 0
            action = probs.argmax()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.float)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            # reward = bad_step_reward_stage_1
            print("Neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            # print("state:")
            # game.print_state()
            print("Номер эпизода происшествия:", episode)
            bad_choise.append(action)
        else:
            # print("Ход нейросети", action + 1, rezult_step)
            if bad_choise != []:
                zero_list_state.append(old_state)
                bad_choise = []
        # elif rezult_step == "Хороший ход!":
        #     reward = good_step_reward_stage_1
        # elif rezult_step == "Хороший ход! Захват!":
        #     reward = good_step_captured_reward_stage_1

        # Нейронка вместо бота
        if old_player_making_step != game.get_player_making_step():
            # game.print_state()
            # take_step_neuronet(game)
            take_step_bot(game)
            # print("Ход противника", )


    if game.get_winner() != None:
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1

    # print("Очередь хода нейронки", neuronet_walker_queue)
    # print("Результат:", game.get_player_winner(), 'k', game.get_winner())
    # print("Номер попытки:", episode)
    # print("Награда/Штраф:", reward)

    # print(episode)

print(count_choisen_zero)
print(count_win_neuronet/episodes_count)

0
0.774


In [ ]:
is_save = True
# is_save = False
if is_save:
    torch.save(model, work_dir+"model5_winner_v15.pt")

**Попытка сохранить нули в файл и на них пообучать**

In [ ]:
print(len(torch.unique(torch.Tensor(zero_list_state))))

ValueError: ignored

In [ ]:
with open(work_dir + "./zero_choise_state", 'wb') as fp:
    pickle.dump(zero_list_state, fp)

In [ ]:
with open(work_dir + "./zero_choise_state", 'rb') as fp:
    zero_list_state_from_file = pickle.load(fp)

In [ ]:
loss = loss_func
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-6, amsgrad=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [54]:

# Убираем нули
episodes_count = 1
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
rezult_step = ""

for episode in range(0, episodes_count):

     for zero_state in zero_list_state_from_file:
        game = Kalah()
        game.set_state(zero_state)
        print(game.get_state())
        # while "Хороший ход!" not in rezult_step:
        # Выбор хода
        probs = model(game.get_state())
        m = Categorical(probs)
        action = m.sample()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.float)
        old_score = game.get_score_player(neuronet_walker_queue)

        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            print("Neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            print("Номер эпизода происшествия:", episode)
            bad_choise.append(action)
            reward = -25
        else:
            reward = game.get_score_player(neuronet_walker_queue) - old_score
            if "Дополнительный ход" in rezult_step:
                reward += 4
            if bad_choise != []:
                zero_list_state.append(old_state)
                bad_choise = []

        # loss = 0.
        alpha = 0.1
        loss = -alpha*reward*m.log_prob(action)

        print(episode, loss, "Номер хода нейронки", neuronet_walker_queue, reward)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

print(count_choisen_zero)
print(count_win_neuronet/episodes_count)

NameError: name 'zero_list_state_from_file' is not defined

In [ ]:
print(count_choisen_zero)
print(count_win_neuronet/episodes_count)